In [1]:
import gym
import torch
import torch.nn as nn
import pickle
import wandb
import pandas as pd
import warnings
import argparse
import numpy as np
import random
from tqdm import tqdm
from collections import deque
from models.lunar_lander_dqn_architecture import lunar_lander_mlp
from rl_utils.stack_frames import stacked_frames_class
from rl_utils.replay_buffer import memory
from rl_utils.replay_buffer import memory_with_curriculum
from rl_utils.optimization import perform_optimization_step

if torch.cuda.device_count() > 1:
    device = torch.device("cuda:"+gpu if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

Using device: cuda


In [2]:
env = gym.make('LunarLander-v2')
obs = env.reset(seed=42)

In [3]:
obs[0]

array([ 0.00229702,  1.4181306 ,  0.2326471 ,  0.3204666 , -0.00265488,
       -0.05269805,  0.        ,  0.        ], dtype=float32)

In [4]:
env.observation_space.shape[0]

8

In [5]:
model = lunar_lander_mlp(env, 64, 64)

In [6]:
model(obs[0])

tensor([-0.1386, -0.0539,  0.0770,  0.0437], grad_fn=<AddBackward0>)

In [40]:
for i in range(200):
    # USing thee environment with random actions
    env = gym.make('LunarLander-v2')
    env.reset()
    rewards = []
    done = False
    lengths = []
    while not done:
        action = env.action_space.sample()
        obs, reward, done, _, info  = env.step(action)
        lengths.append(len(obs))
        rewards.append(reward)
    print(min(lengths), max(lengths))

/home/mario.cantero/.conda/envs/rl_research/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8


In [42]:
env.reset()

(array([ 0.00680161,  1.4063674 ,  0.6889165 , -0.20235749, -0.00787459,
        -0.1560499 ,  0.        ,  0.        ], dtype=float32),
 {})

In [46]:
a, b,c ,d, e = env.step(0)

In [47]:
a

array([ 0.02720795,  1.3891824 ,  0.68803257, -0.28137437, -0.03099762,
       -0.15412304,  0.        ,  0.        ], dtype=float32)

In [8]:
len(rewards)

115

In [16]:
obs

array([-0.9374373 ,  0.16288441, -1.2419778 , -0.06932734,  0.79287654,
       -0.63573176,  0.        ,  1.        ], dtype=float32)

In [9]:
done

True

In [13]:
device = "cpu"
action, eps = model.select_action(env, obs, 0.1, 1, 0.1, 1, device)

In [15]:
action.item()

3